In [75]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import albumentations as A
import os
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

In [77]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

joint_mapping = {
    11: "left_shoulder", 12: "right_shoulder",
    13: "left_elbow", 14: "right_elbow",
    15: "left_wrist", 16: "right_wrist",
    23: "left_hip", 24: "right_hip",
    25: "left_knee", 26: "right_knee",
    27: "left_ankle", 28: "right_ankle"
}

In [79]:
def augment_image(image):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.GaussianBlur(p=0.2)
    ])
    return transform(image=image)['image']

In [81]:
def extract_landmarks(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    if results.pose_landmarks:
        landmarks = {}
        for idx, lm in enumerate(results.pose_landmarks.landmark):
            if idx in joint_mapping:
                landmarks[joint_mapping[idx]] = [lm.x, lm.y, lm.z]
        return landmarks
    return None

In [83]:
def normalize_landmarks(landmarks):
    if not landmarks:
        return None
    min_x = min(landmarks[joint][0] for joint in landmarks)
    min_y = min(landmarks[joint][1] for joint in landmarks)
    max_x = max(landmarks[joint][0] for joint in landmarks)
    max_y = max(landmarks[joint][1] for joint in landmarks)
    for joint in landmarks:
        landmarks[joint][0] = (landmarks[joint][0] - min_x) / (max_x - min_x)
        landmarks[joint][1] = (landmarks[joint][1] - min_y) / (max_y - min_y)
    return landmarks

In [85]:
dataset = {}
dataset_path = "dataset"
yoga_classes = os.listdir(dataset_path)
valid_extensions = (".jpg", ".jpeg", ".png")

for pose_name in yoga_classes:
    pose_folder = os.path.join(dataset_path, pose_name)
    dataset[pose_name] = []
    for img_name in os.listdir(pose_folder):
        if not img_name.lower().endswith(valid_extensions):
            continue
        img_path = os.path.join(pose_folder, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue
        image = augment_image(image)
        landmarks = extract_landmarks(image)
        landmarks = normalize_landmarks(landmarks)
        if landmarks:
            dataset[pose_name].append(landmarks)

In [134]:
def draw_feedback(frame, pose_name, feedback):
    cv2.putText(frame, f"Pose: {pose_name}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
    y0, dy = 70, 25
    for i, line in enumerate(feedback.split(',')):
        y = y0 + i * dy
        cv2.putText(frame, line.strip(), (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv2.LINE_AA)
    return frame

In [89]:
with open("pose_dataset.json", "w") as f:
    json.dump(dataset, f, indent=4)

In [91]:
X, y = [], []
class_labels = {pose_name: i for i, pose_name in enumerate(yoga_classes)}

for pose_name, samples in dataset.items():
    for sample in samples:
        landmarks = [sample[joint] for joint in joint_mapping.values() if joint in sample]
        if landmarks:
            X.append(np.array(landmarks).flatten())
            y.append(class_labels[pose_name])

X, y = np.array(X), to_categorical(y, num_classes=len(yoga_classes))

np.savez("pose_data.npz", X=X, y=y)

In [93]:
data = np.load("pose_data.npz")
X, y = data["X"], data["y"]

In [120]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input

num_classes = len(yoga_classes)

model = Sequential([
    Input(shape=(len(joint_mapping) * 3,)),  # Input Layer
    
    Dense(512, activation='relu'),  # More neurons for better feature extraction
    BatchNormalization(),
    Dropout(0.4),

    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(num_classes, activation='softmax')  # Output layer
])

# Adam optimizer with a reduced learning rate for better convergence
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

# Train with early stopping to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

model.save("optimized_pose_classifier.h5")
pose_model = load_model("optimized_pose_classifier.h5")


Epoch 1/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0193 - loss: 4.9173 - val_accuracy: 0.0064 - val_loss: 4.5838
Epoch 2/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.0760 - loss: 4.2652 - val_accuracy: 0.0354 - val_loss: 4.1250
Epoch 3/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1279 - loss: 3.8593 - val_accuracy: 0.0934 - val_loss: 3.7815
Epoch 4/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1640 - loss: 3.5860 - val_accuracy: 0.1058 - val_loss: 3.5588
Epoch 5/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.1948 - loss: 3.4031 - val_accuracy: 0.1337 - val_loss: 3.3462
Epoch 6/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2170 - loss: 3.2542 - val_accuracy: 0.1450 - val_loss: 3.1865
Epoch 7/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2376 - loss: 3.1654 - val_accuracy: 0.1643 - val_loss: 3.0663
Epoch 8/100
233/233 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2549 - loss: 3.0454 - val_accu

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
import tensorflow_gnn as tfgnn

num_classes = len(yoga_classes)

# Graph input structure (Body joints as nodes)
input_shape = (len(joint_mapping) * 3,)

# Define GCN Layer
def GCNLayer(units):
    return tfgnn.keras.layers.GraphUpdate(
        node_set_updates={
            "joints": tfgnn.keras.layers.NodeSetUpdate(
                {
                    "joints->joints": tfgnn.keras.layers.SimpleConv(
                        num_message_passing=2,
                        units=units,
                        activation="relu"
                    )
                }
            )
        }
    )

# Model architecture
inputs = Input(shape=input_shape)

# GCN Layers
x = GCNLayer(128)(inputs)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = GCNLayer(64)(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

# MLP Classifier
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = Dense(32, activation='relu')(x)
x = BatchNormalization()(x)

outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train with Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

model.save("gcn_pose_classifier.h5")
pose_model_g = tf.keras.models.load_model("gcn_pose_classifier.h5")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [128]:
def predict_pose(landmarks):
    landmarks = np.array(landmarks).flatten()  # Ensure a 1D vector
    landmarks = np.expand_dims(landmarks, axis=0)  # Add batch dimension
    return pose_model.predict(landmarks)


In [138]:
def provide_feedback(landmarks, prediction, class_labels):
    pose_index = np.argmax(prediction)
    pose_name = list(class_labels.keys())[pose_index]
    feedback = "Good Pose!"

    # If confidence is low, provide corrections
    if np.max(prediction) < 0.8:
        feedback = "Adjust: "
        for joint in joint_mapping.values():
            if joint in landmarks:
                x, y, _ = landmarks[joint]
                if x < 0.4:
                    feedback += f"Move {joint} right, "
                elif x > 0.6:
                    feedback += f"Move {joint} left, "
                if y < 0.4:
                    feedback += f"Move {joint} down, "
                elif y > 0.6:
                    feedback += f"Move {joint} up, "
    
    return pose_name, feedback.strip(', ')


In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    landmarks = extract_landmarks(frame)
    landmarks = normalize_landmarks(landmarks)
    if landmarks:
        prediction = predict_pose([landmarks[joint] for joint in joint_mapping.values() if joint in landmarks])
        pose_name, feedback = provide_feedback(landmarks, prediction, class_labels)
        frame = draw_feedback(frame, pose_name, feedback)
    cv2.imshow('Yoga Pose Correction', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━